In [1]:
from gensim import corpora, models, similarities
import pandas as pd

reviews = pd.read_json('Reviews20160219.json')

In [2]:
reviews.shape

(27080, 5)

In [3]:
reviews.head()

,ReviewerExperience,date,name,rating,review
0,382,2012-12-10,Tian Tian Hainanese Chicken Rice,5,I don't even like 海南雞飯 based on past attempts....
1,44,2014-06-18,Tian Tian Hainanese Chicken Rice,5,"Pros: chicken rice is delicious Cons: Hot, no ..."
2,285,2013-06-18,Tian Tian Hainanese Chicken Rice,4,The rice is freakin incredible!!! Wow I was su...
3,112,2013-02-04,Tian Tian Hainanese Chicken Rice,2,DUDE! WHAT HAPPENED TO THIS PLACE?! I kept hea...
4,5,2014-05-31,Tian Tian Hainanese Chicken Rice,4,The Chicken Rice is Singapore's national dish ...


In [4]:
rev_text_norm = pd.Series(reviews.drop_duplicates().review)
rev_text_norm.size

20334

In [5]:
import re, unicodedata
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

In [6]:
def remove_non_ascii(string):
    ### remove unreadable unicode characters
    return unicodedata.normalize('NFKD', string).encode('ascii','ignore')
    #return string.decode('unicode_escape').encode('ascii','ignore')

def review_to_words(raw_review):
    ascii_only = remove_non_ascii(raw_review)
    letters_only = re.sub("[^a-zA-Z]", " ", ascii_only)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    #return( " ".join( meaningful_words ))
    return meaningful_words

In [7]:
texts = [review_to_words(x) for x in rev_text_norm]

In [8]:
texts[0:5]

[['even',
  'like',
  'based',
  'past',
  'attempts',
  'thought',
  'made',
  'obligatory',
  'visit',
  'tian',
  'tian',
  'expecting',
  'lone',
  'dissenter',
  'anthony',
  'bourdain',
  'photo',
  'plastered',
  'stall',
  'check',
  'huge',
  'queue',
  'anything',
  'resembling',
  'lunch',
  'hour',
  'check',
  'serving',
  'ware',
  'looks',
  'like',
  'seen',
  'minor',
  'battle',
  'long',
  'forgotten',
  'war',
  'check',
  'good',
  'really',
  'good',
  'especially',
  'chicken',
  'stock',
  'combination',
  'chicken',
  'rice',
  'light',
  'still',
  'meaty',
  'enough',
  'reaching',
  'spice',
  'best',
  'ever',
  'yes',
  'overrated',
  'probably',
  'since',
  'cannot',
  'cure',
  'cancer',
  'world',
  'hunger',
  'simultaneously',
  'still',
  'absolutely',
  'come',
  'peak',
  'visiting',
  'singapore',
  'absolutely'],
 ['pros',
  'chicken',
  'rice',
  'delicious',
  'cons',
  'hot',
  'c',
  'waiting',
  'line'],
 ['rice',
  'freakin',
  'incredible

In [9]:
>>> # remove words that appear only once
>>> from collections import defaultdict
>>> frequency = defaultdict(int)
>>> for text in texts:
>>>     for token in text:
>>>         frequency[token] += 1

In [10]:
token

'couples'

In [11]:
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]
#from pprint import pprint   # pretty-printer
#pprint(texts)

In [12]:
dictionary = corpora.Dictionary(texts)

In [13]:
print(dictionary)

Dictionary(22901 unique tokens: [u'gai', u'woods', u'clotted', u'hanging', u'woody']...)


In [14]:
print(dictionary.token2id)

{u'gai': 418, u'woods': 2951, u'clotted': 3045, u'hanging': 2906, u'woody': 12705, u'trawling': 11598, u'comically': 16402, u'localized': 16273, u'chatter': 8837, u'originality': 6195, u'refunding': 17633, u'fattiness': 13254, u'bringing': 3936, u'broiler': 16636, u'wooden': 7060, u'wholemeal': 3209, u'wednesday': 2095, u'broiled': 930, u'capsicum': 9961, u'stereotypical': 17109, u'bbqs': 10470, u'sooth': 18840, u'scraped': 2944, u'errors': 9012, u'cooking': 2376, u'succumb': 13643, u'ching': 20353, u'china': 203, u'wagyu': 3822, u'affiliated': 17365, u'doldrums': 19556, u'kids': 2652, u'uplifting': 14116, u'naturel': 15734, u'robata': 20117, u'climbed': 12027, u'preparazzi': 21489, u'controversy': 6492, u'spotty': 12578, u'golden': 1425, u'stern': 12909, u'catchy': 14987, u'music': 5061, u'therefore': 1090, u'mitsuya': 21107, u'excl': 12304, u'caneles': 21858, u'circumstances': 14632, u'intake': 11138, u'morally': 17817, u'locked': 8999, u'busyness': 18371, u'coulda': 20705, u'wang': 

In [15]:
# test bag of words
new_vec = dictionary.doc2bow(review_to_words(u"Chicken rice tastes so damn good so"))
print(new_vec) 

[(12, 1), (36, 1), (52, 1), (1522, 1), (1609, 1)]


In [16]:
corpus = [dictionary.doc2bow(text) for text in texts] # all sentences converted into bag of words
#>>> corpora.MmCorpus.serialize('deerwester.mm', corpus) # store to disk, for later use
#>>> print(corpus)

In [17]:
tfidf = models.TfidfModel(corpus)

In [18]:
corpus_tfidf = tfidf[corpus]
#for doc in corpus_tfidf:
#    print(doc)

In [20]:
>>> #### SERIALIZATION OF TRANSFORMATIONS to save memory space
>>> lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=50) # initialize an LSI transformation
>>> corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
>>> # transformed our tf-ldf corpus via latent semantic indexing into 2-D space (num_topics = 2)
>>> lsi.print_topics(10)

[(0,
  u'0.134*"chicken" + 0.115*"food" + 0.114*"rice" + 0.112*"good" + 0.108*"place" + 0.100*"really" + 0.096*"like" + 0.095*"great" + 0.093*"one" + 0.091*"service"'),
 (1,
  u'0.340*"chicken" + 0.331*"rice" + 0.298*"ramen" + -0.191*"coffee" + 0.189*"soup" + -0.174*"pizza" + 0.167*"noodles" + 0.154*"fried" + 0.126*"pork" + 0.102*"stall"'),
 (2,
  u'-0.798*"ramen" + 0.280*"chicken" + 0.234*"rice" + -0.133*"broth" + 0.122*"crab" + 0.110*"thai" + -0.103*"noodles" + -0.089*"ippudo" + 0.082*"fried" + -0.082*"japanese"'),
 (3,
  u'-0.707*"crab" + 0.285*"chicken" + -0.249*"chili" + 0.199*"rice" + -0.199*"pepper" + -0.146*"crabs" + -0.110*"chilli" + 0.110*"coffee" + -0.108*"seafood" + -0.100*"black"'),
 (4,
  u'-0.666*"sushi" + 0.242*"coffee" + -0.217*"sashimi" + -0.208*"japanese" + -0.130*"salmon" + 0.109*"pizza" + 0.103*"ramen" + -0.102*"fish" + 0.101*"chicken" + -0.100*"quality"'),
 (5,
  u'0.703*"pizza" + -0.342*"coffee" + 0.144*"pizzas" + 0.139*"chicken" + 0.129*"pasta" + 0.126*"crust" +

In [21]:
#### SERIALIZATION OF TRANSFORMATIONS to save memory space
lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=50) # initialize an LDA transformation with tf-idf

lda_model.print_topics(10)

[(24,
  u'0.016*meatballs + 0.008*hainanese + 0.008*ivy + 0.008*lava + 0.007*nam + 0.006*keeping + 0.006*malaysian + 0.006*lobby + 0.006*combos + 0.006*writing'),
 (45,
  u'0.017*otak + 0.012*ban + 0.009*overlooking + 0.008*fashioned + 0.008*jewel + 0.008*sua + 0.007*apologetic + 0.007*oriole + 0.007*waved + 0.007*conveniently'),
 (0,
  u'0.022*terminal + 0.020*changi + 0.015*rendang + 0.014*elevator + 0.013*hk + 0.012*pisang + 0.012*airport + 0.011*superb + 0.010*scones + 0.010*attracted'),
 (7,
  u'0.013*ramen + 0.011*hoon + 0.011*bee + 0.010*stall + 0.009*kee + 0.008*kopitiam + 0.008*court + 0.008*sells + 0.007*chang + 0.007*dimsum'),
 (41,
  u'0.015*hokkien + 0.011*mee + 0.010*espresso + 0.007*coffees + 0.007*italian + 0.006*chee + 0.005*natural + 0.005*grey + 0.005*dough + 0.005*hawkers'),
 (19,
  u'0.053*ta + 0.030*junction + 0.013*ignored + 0.013*imperial + 0.013*yu + 0.012*limp + 0.009*renovations + 0.009*kill + 0.008*auntie + 0.008*liu'),
 (28,
  u'0.021*rainbow + 0.015*maison

In [22]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      67.517579        1       1 -0.357664 -0.160474
12      7.099466        1       2 -0.294999 -0.078287
1       4.098214        1       3 -0.275984 -0.030085
43      2.137024        1       4 -0.206051  0.010949
14      2.051961        1       5 -0.202033  0.065735
7       1.214834        1       6 -0.140243  0.012304
41      1.212860        1       7 -0.092120  0.223620
8       1.170621        1       8 -0.085344  0.122883
17      1.043207        1       9 -0.122559  0.066349
4       0.874697        1      10 -0.086956  0.061215
24      0.854167        1      11 -0.081624  0.034596
32      0.626188        1      12 -0.080378 -0.007743
49      0.485722        1      13  0.044185  0.034956
18      0.459366        1      14 -0.023037  0.056147
10      0.437689        1      15  0.002707  0.052779
3       0.415414        1      16 -0.028620 -0.027647
45      0.356581        1      17 -0.012672 -0.018043
42      0.352041        1      18  0.046828 -0.002159
47      0.348595        1      19  0.022464  0.008632
22      0.341281        1      20  0.045658  0.016880
15      0.336046        1      21  0.009926 -0.033119
37      0.335822        1      22  0.009339 -0.024722
38      0.335452        1      23  0.045122  0.010774
20      0.329430        1      24  0.030880 -0.016453
29      0.320219        1      25  0.066314 -0.017223
11      0.314019        1      26  0.048332 -0.014134
31      0.311538        1      27  0.024036 -0.014506
35      0.309834        1      28  0.022105 -0.008166
27      0.267691        1      29  0.056168 -0.014696
36      0.265945        1      30  0.095529 -0.006540
34      0.248565        1      31  0.070913 -0.019711
39      0.243431        1      32  0.066720 -0.021082
2       0.235231        1      33  0.067808 -0.009907
25      0.233400        1      34  0.046460 -0.014506
40      0.218046        1      35  0.045149 -0.023345
0       0.210006        1      36  0.062534 -0.007964
19      0.203963        1      37  0.068970 -0.011873
33      0.201192        1      38  0.069859 -0.025387
48      0.199984        1      39  0.071634 -0.010176
6       0.196899        1      40  0.080312 -0.014515
21      0.191258        1      41  0.084039 -0.012652
16      0.186930        1      42  0.070877 -0.020511
13      0.186218        1      43  0.083043 -0.014634
30      0.171093        1      44  0.077997 -0.017780
28      0.156768        1      45  0.079660 -0.013238
44      0.152821        1      46  0.097852 -0.012907
23      0.149424        1      47  0.102239 -0.011202
26      0.143951        1      48  0.081413 -0.011766
46      0.140119        1      49  0.092827 -0.016965
5       0.107199        1      50  0.100383 -0.013701, topic_info=      Category         Freq           Term        Total  loglift  logprob
term                                                                     
21950  Default  2700.000000           view  2700.000000  30.0000  30.0000
20345  Default  1354.000000            dim  1354.000000  29.0000  29.0000
22321  Default  2338.000000          stall  2338.000000  28.0000  28.0000
4572   Default  2694.000000          laksa  2694.000000  27.0000  27.0000
7938   Default  4906.000000           food  4906.000000  26.0000  26.0000
18314  Default  3795.000000          great  3795.000000  25.0000  25.0000
16000  Default  2943.000000        noodles  2943.000000  24.0000  24.0000
22708  Default  5154.000000        chicken  5154.000000  23.0000  23.0000
10749  Default  4550.000000           good  4550.000000  22.0000  22.0000
16278  Default  4699.000000          place  4699.000000  21.0000  21.0000
1489   Default  4766.000000         coffee  4766.000000  20.0000  20.0000
9562   Default   809.000000             ku   809.000000  19.0000  19.0000
117    Default  3795.000000        service  3795.000000  18.0000  18.0000
13121  Default  1429.000000         sta

In [40]:
for doc in corpus_tfidf[0:50]:
    print(doc)

[(0, 0.08336083604652836), (1, 0.07627094834223797), (2, 0.04617449205619288), (3, 0.20926294063264536), (4, 0.1475290943352146), (5, 0.18306243590789284), (6, 0.18245069651012585), (7, 0.09754330338236454), (8, 0.16616639466254768), (9, 0.18245069651012585), (10, 0.1508556521134043), (11, 0.0843473557273314), (12, 0.16870073567130628), (13, 0.09958069477271554), (14, 0.0801068740519404), (15, 0.1011766349398468), (16, 0.25008250813958505), (17, 0.045350701155222904), (18, 0.03355069895421433), (19, 0.04563976580954829), (20, 0.07545128130144359), (21, 0.09202331833862858), (22, 0.10721015115433935), (23, 0.06063704062787484), (24, 0.0687789768682017), (25, 0.05719426873872109), (26, 0.20185656177835165), (27, 0.053644366758667385), (28, 0.059939229707935124), (29, 0.12968401077731265), (30, 0.18935715648085055), (31, 0.06461232598448433), (32, 0.12393917702441148), (33, 0.07160158478372249), (34, 0.17958719102674603), (35, 0.15317659332736142), (36, 0.11494449639992793), (37, 0.081434